In [1]:
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery
from google.oauth2 import service_account
# Cargar datos originales
dfDetalleCompra = pd.read_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\Data\detalle_compra.csv')
dfSorteo = pd.read_csv(r"C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\Data\sorteo.csv", dtype=str)

# Convertir tipos de datos necesarios
dfSorteo["clave_edicion_producto"] = dfSorteo["clave_edicion_producto"].astype(int)
dfSorteo["tipo_producto"] = dfSorteo["tipo_producto"].dropna().astype(float).astype(int)
dfSorteo["numero_sorteo"] = dfSorteo["numero_sorteo"].dropna().astype(float).astype(int)

# Convertir fecha_fin a datetime
dfSorteo['fecha_fin'] = pd.to_datetime(dfSorteo['fecha_fin'])

# Obtener fecha actual y calcular fecha hace 2 años
fecha_actual = datetime.now()
fecha_dos_anios_atras = fecha_actual - timedelta(days=730)  # 2 años = 730 días

# Filtrar sorteos: fecha_fin < fecha_actual Y fecha_fin > fecha_actual - 2 años
dfSorteoFiltrado = dfSorteo.loc[
    (dfSorteo["fecha_fin"] < fecha_actual) & 
    (dfSorteo["fecha_fin"] > fecha_dos_anios_atras)
].copy()

# Crear columna item_completo
dfSorteoFiltrado['item_completo'] = dfSorteoFiltrado['desc_sorteo'] + ' ' + dfSorteoFiltrado['numero_sorteo'].astype(str)

# Merge detalle_compra con sorteo filtrado (inner join para solo mantener sorteos filtrados)
dfDetalleCompraFiltrado = pd.merge(
    dfDetalleCompra, 
    dfSorteoFiltrado[['clave_edicion_producto', 'item_completo', "tipo_producto", "numero_sorteo"]], 
    on='clave_edicion_producto', 
    how='inner'
)
dfDetalleCompraFiltrado.drop(columns=["item_completo","tipo_producto","numero_sorteo"],inplace=True)
# Guardar archivos filtrados
dfDetalleCompraFiltrado.to_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\H3Canibalization\detalle_compra_2_años.csv', index=False)
dfSorteoFiltrado.to_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\H3Canibalization\sorteo_2_años.csv', index=False)

print(f"✓ Sorteos filtrados: {len(dfSorteoFiltrado)}")
print(f"✓ Registros detalle compra filtrados: {len(dfDetalleCompraFiltrado)}")

✓ Sorteos filtrados: 128
✓ Registros detalle compra filtrados: 1872214


In [2]:
dfDetalleCompraFiltrado

,clave_detalle_compra,agrupador_interno_compra,clave_persona,clave_rotulo,clave_edicion_producto,fecha_rotulacion_finalizacion,clave_boleto_unico,clave_boleto_intranet,numero_boleto
0,2413005,3319547,139436,501393,509,2024-10-27 16:10:55.669195,509005330.0,5897152,5330
1,2414221,3320431,103220,152563,504,2024-10-27 21:51:16.606613,504007388.0,5820586,7388
2,2414157,3320409,70370,385977,504,2024-10-27 21:46:36.116023,504003000.0,5836156,3000
3,2414946,3320900,304559,512273,504,2024-10-28 02:01:57.851704,504005236.0,5819407,5236
4,2420915,3328650,93039,137421,504,2024-10-29 08:36:50.122518,504006246.0,5837472,6246
...,...,...,...,...,...,...,...,...,...
1872209,4282622,5979448,149157,253887,542,2025-09-10 18:59:07.391205,542010701.0,6963687,10701
1872210,4285545,5982085,373781,598886,542,2025-09-11 09:30:58.810907,542005781.0,6925002,5781
1872211,4285324,5981814,43356,257731,542,2025-09-11 08:40:39.998329,542010146.0,6974618,10146
1872212,4285528,5982060,340637,557047,542,2025-09-11 09:31:29.392858,542000788.0,6974466,788


In [5]:
import pandas as pd
import json
from pathlib import Path

# -------------------------
# Cargar datos
# -------------------------
df_detalle = pd.read_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\H3Canibalization\detalle_compra_SMS.csv')
df_sorteo = pd.read_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\H3Canibalization\sorteo_SMS.csv')

# Parsear fechas
def parse_fechas(x):
    for fmt in ("%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S"):
        try:
            return pd.to_datetime(x, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

df_detalle["fecha_rotulacion_finalizacion"] = df_detalle["fecha_rotulacion_finalizacion"].apply(parse_fechas)
df_sorteo['fecha_inicio'] = pd.to_datetime(df_sorteo['fecha_inicio'])
df_sorteo['fecha_fin'] = pd.to_datetime(df_sorteo['fecha_fin'])
df_sorteo['fecha_celebracion'] = pd.to_datetime(df_sorteo['fecha_celebracion'])

# -------------------------
# Enriquecer detalle con info de sorteo
# -------------------------
df_enriq = df_detalle.merge(
    df_sorteo[['clave_edicion_producto', 'tipo_producto', 'numero_sorteo', 'desc_sorteo', 
               'fecha_inicio', 'fecha_fin', 'fecha_celebracion']].drop_duplicates('clave_edicion_producto'),
    on='clave_edicion_producto',
    how='left'
)

# Filtrar solo registros válidos
df_enriq = df_enriq[
    df_enriq['fecha_rotulacion_finalizacion'].notna() &
    df_enriq['tipo_producto'].notna() &
    df_enriq['numero_sorteo'].notna()
].copy()

# -------------------------
# Determinar última edición por producto (max numero_sorteo)
# -------------------------
last_edition = (
    df_sorteo[['tipo_producto', 'numero_sorteo', 'clave_edicion_producto', 'fecha_inicio', 'fecha_fin']]
    .sort_values(['tipo_producto', 'numero_sorteo'], ascending=[True, False])
    .groupby('tipo_producto', as_index=False)
    .first()
    .rename(columns={
        'numero_sorteo': 'numero_sorteo_ultima',
        'clave_edicion_producto': 'clave_edicion_producto_ultima',
        'fecha_inicio': 'ultima_fecha_inicio',
        'fecha_fin': 'ultima_fecha_fin',
    })
)

df_enriq = df_enriq.merge(last_edition, on='tipo_producto', how='left')

# -------------------------
# Marcar si cada compra es de la última edición
# -------------------------
df_enriq['es_ultima_edicion'] = df_enriq['numero_sorteo'] == df_enriq['numero_sorteo_ultima']

# Marcar si está en semana de cierre (últimos 7 días de la edición)
df_enriq['semana_cierre_ini'] = df_enriq['fecha_fin'] - pd.Timedelta(days=7)
df_enriq['en_semana_cierre'] = (
    (df_enriq['fecha_rotulacion_finalizacion'] >= df_enriq['semana_cierre_ini']) &
    (df_enriq['fecha_rotulacion_finalizacion'] <= df_enriq['fecha_fin'])
)

# Marcar si está en pre-cierre (antes de la semana de cierre, dentro de la edición)
df_enriq['en_pre_cierre'] = (
    (df_enriq['fecha_rotulacion_finalizacion'] >= df_enriq['fecha_inicio']) &
    (df_enriq['fecha_rotulacion_finalizacion'] < df_enriq['semana_cierre_ini'])
)

# -------------------------
# Función para contar boletos
# -------------------------
def count_tickets(df_group):
    if 'numero_boleto' in df_group.columns and df_group['numero_boleto'].notna().any():
        return df_group['numero_boleto'].nunique()
    return len(df_group)

# -------------------------
# Agregación por cliente × producto
# -------------------------
rollup_list = []

for (persona, tipo), df_grp in df_enriq.groupby(['clave_persona', 'tipo_producto']):
    
    # --- HISTÓRICO (ediciones anteriores a la última) ---
    df_hist = df_grp[~df_grp['es_ultima_edicion']]
    hist_pre = count_tickets(df_hist[df_hist['en_pre_cierre']]) if not df_hist.empty else 0
    hist_cierre = count_tickets(df_hist[df_hist['en_semana_cierre']]) if not df_hist.empty else 0
    hist_ediciones = df_hist['numero_sorteo'].nunique() if not df_hist.empty else 0
    
    # --- ÚLTIMA EDICIÓN ---
    df_ult = df_grp[df_grp['es_ultima_edicion']]
    ult_pre = count_tickets(df_ult[df_ult['en_pre_cierre']]) if not df_ult.empty else 0
    ult_cierre = count_tickets(df_ult[df_ult['en_semana_cierre']]) if not df_ult.empty else 0
    compro_ultima = (ult_pre + ult_cierre) > 0
    
    # Datos de la última edición del producto
    if not df_ult.empty:
        ultima_ini = df_ult['ultima_fecha_inicio'].iloc[0]
        ultima_fin = df_ult['ultima_fecha_fin'].iloc[0]
        numero_sorteo_ultima = df_ult['numero_sorteo_ultima'].iloc[0]
        clave_edicion_ultima = df_ult['clave_edicion_producto_ultima'].iloc[0]
    else:
        row_last = last_edition[last_edition['tipo_producto'] == tipo]
        if not row_last.empty:
            ultima_ini = row_last['ultima_fecha_inicio'].iloc[0]
            ultima_fin = row_last['ultima_fecha_fin'].iloc[0]
            numero_sorteo_ultima = row_last['numero_sorteo_ultima'].iloc[0]
            clave_edicion_ultima = row_last['clave_edicion_producto_ultima'].iloc[0]
        else:
            ultima_ini = pd.NaT
            ultima_fin = pd.NaT
            numero_sorteo_ultima = None
            clave_edicion_ultima = None
    
    # --- OTROS SORTEOS (tipo diferente) en ventana de última edición ---
    otros_bol_cierre = 0
    otros_bol_fuera = 0
    otros_detalle = []
    
    if pd.notna(ultima_ini) and pd.notna(ultima_fin):
        df_cliente = df_enriq[df_enriq['clave_persona'] == persona]
        en_ventana = (
            (df_cliente['fecha_rotulacion_finalizacion'] >= ultima_ini) &
            (df_cliente['fecha_rotulacion_finalizacion'] <= ultima_fin)
        )
        df_otros = df_cliente[en_ventana & (df_cliente['tipo_producto'] != tipo)]
        
        if not df_otros.empty:
            cierre_ini_obj = ultima_fin - pd.Timedelta(days=7)
            df_otros['en_cierre_objetivo'] = (
                (df_otros['fecha_rotulacion_finalizacion'] >= cierre_ini_obj) &
                (df_otros['fecha_rotulacion_finalizacion'] <= ultima_fin)
            )
            
            for (tp2, ns2), df_g2 in df_otros.groupby(['tipo_producto', 'numero_sorteo']):
                bol_cierre = count_tickets(df_g2[df_g2['en_cierre_objetivo']])
                bol_fuera = count_tickets(df_g2[~df_g2['en_cierre_objetivo']])
                
                otros_bol_cierre += bol_cierre
                otros_bol_fuera += bol_fuera
                
                # Convertir a tipos nativos de Python para JSON
                otros_detalle.append({
                    'tipo_producto': str(tp2),
                    'numero_sorteo': int(ns2) if pd.notna(ns2) else None,
                    'id_sorteo': f"{str(tp2)}-{int(ns2)}" if pd.notna(ns2) else f"{str(tp2)}-None",
                    'clave_edicion_producto': int(df_g2['clave_edicion_producto'].iloc[0]) if pd.notna(df_g2['clave_edicion_producto'].iloc[0]) else None,
                    'boletos_en_semana_cierre_objetivo': int(bol_cierre),
                    'boletos_fuera_semana_cierre_objetivo': int(bol_fuera),
                })
    
    otros_tipos_distintos = len(set(r['tipo_producto'] for r in otros_detalle))
    
    # --- DETALLE HISTÓRICO por edición ---
    hist_detalle = []
    if not df_hist.empty:
        for ns_h, df_h in df_hist.groupby('numero_sorteo'):
            # Convertir a tipos nativos de Python para JSON
            hist_detalle.append({
                'numero_sorteo': int(ns_h) if pd.notna(ns_h) else None,
                'tipo_producto': str(tipo),
                'id_sorteo': f"{str(tipo)}-{int(ns_h)}" if pd.notna(ns_h) else f"{str(tipo)}-None",
                'clave_edicion_producto': int(df_h['clave_edicion_producto'].iloc[0]) if pd.notna(df_h['clave_edicion_producto'].iloc[0]) else None,
                'boletos_pre_cierre': int(count_tickets(df_h[df_h['en_pre_cierre']])),
                'boletos_semana_cierre': int(count_tickets(df_h[df_h['en_semana_cierre']])),
                'fecha_fin_edicion': df_h['fecha_fin'].iloc[0].strftime('%Y-%m-%d') if pd.notna(df_h['fecha_fin'].iloc[0]) else None
            })
    
    # --- CLASIFICACIÓN DE CANIBALIZACIÓN ---
    if not compro_ultima and (otros_bol_cierre + otros_bol_fuera) > 0:
        cani = 'PERDIDO_A_OTRO_EN_CIERRE' if otros_bol_cierre > 0 else 'PERDIDO_A_OTRO_FUERA_CIERRE'
    elif not compro_ultima:
        cani = 'CHURN_TOTAL'
    elif compro_ultima and (otros_bol_cierre + otros_bol_fuera) > 0:
        cani = 'RETENIDO_Y_OTROS'
    else:
        cani = 'RETENIDO'
    
    # --- GUARDAR RESULTADO ---
    rollup_list.append({
        'clave_persona': int(persona) if pd.notna(persona) else None,
        'tipo_producto': float(tipo) if pd.notna(tipo) else None,
        'numero_sorteo_ultima': int(numero_sorteo_ultima) if pd.notna(numero_sorteo_ultima) else None,
        'clave_edicion_producto_ultima': int(clave_edicion_ultima) if pd.notna(clave_edicion_ultima) else None,
        'ultima_fecha_inicio': ultima_ini.strftime('%Y-%m-%d') if pd.notna(ultima_ini) else None,
        'ultima_fecha_fin': ultima_fin.strftime('%Y-%m-%d') if pd.notna(ultima_fin) else None,
        'ultima_semana_cierre_ini': (ultima_fin - pd.Timedelta(days=7)).strftime('%Y-%m-%d') if pd.notna(ultima_fin) else None,
        'ultima_semana_cierre_fin': ultima_fin.strftime('%Y-%m-%d') if pd.notna(ultima_fin) else None,
        'hist_boletos_pre_cierre': int(hist_pre),
        'hist_boletos_semana_cierre': int(hist_cierre),
        'hist_ediciones_distintas': int(hist_ediciones),
        'ultima_boletos_pre_cierre': int(ult_pre),
        'ultima_boletos_semana_cierre': int(ult_cierre),
        'compro_ultima_edicion': bool(compro_ultima),
        'otros_boletos_en_semana_cierre_del_producto': int(otros_bol_cierre),
        'otros_boletos_fuera_semana_cierre_del_producto': int(otros_bol_fuera),
        'otros_tipos_producto_distintos': int(otros_tipos_distintos),
        'canibalizacion_clase': cani,
        'hist_detalle_por_edicion_json': json.dumps(hist_detalle, ensure_ascii=False),
        'otros_detalle_en_ventana_ultima_json': json.dumps(otros_detalle, ensure_ascii=False),
    })

# -------------------------
# Crear DataFrame final
# -------------------------
rollup_df = pd.DataFrame(rollup_list)

# Guardar
rollup_df.to_csv(r'C:\Sam\Python\ScriptsMaquinaVirtual\MLSiteConversion\H3Canibalization\rollup_canibalizacion.csv', index=False)
print(f"✓ Archivo generado: {len(rollup_df)} registros")

KeyboardInterrupt: 

In [ ]:
# -------------------------
# Preparar datos para BigQuery
# -------------------------

# Convertir columnas de fecha de string a datetime
columnas_fecha = [
    'ultima_fecha_inicio',
    'ultima_fecha_fin', 
    'ultima_semana_cierre_ini',
    'ultima_semana_cierre_fin'
]

for col in columnas_fecha:
    rollup_df[col] = pd.to_datetime(rollup_df[col])

# -------------------------
# Configurar BigQuery
# -------------------------

# Credenciales
credentials = service_account.Credentials.from_service_account_file(
    r'C:\ruta\a\tu\llave.json'  # PLACEHOLDER: Cambia esta ruta
)

# Cliente BigQuery
client = bigquery.Client(
    credentials=credentials,
    project='tu-proyecto-id'  # PLACEHOLDER: Cambia el ID del proyecto
)

# Configuración de la tabla
dataset_id = 'tu_dataset'  # PLACEHOLDER: Cambia el nombre del dataset
table_id = 'rollup_canibalizacion'  # PLACEHOLDER: Cambia si quieres otro nombre
full_table_id = f"{client.project}.{dataset_id}.{table_id}"

# -------------------------
# Configurar job (opcional: eliminar y reemplazar tabla)
# -------------------------

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si existe
)

# -------------------------
# Cargar a BigQuery
# -------------------------

print(f"Cargando {len(rollup_df)} registros a {full_table_id}...")

job = client.load_table_from_dataframe(
    rollup_df,
    full_table_id,
    job_config=job_config
)

job.result()  # Esperar a que termine

print(f"✓ Datos cargados exitosamente a BigQuery: {full_table_id}")
print(f"  Total filas: {job.output_rows}")